# Transformer: 언어 처리를 위한 거대한 공장 (Factory)

**Transformer**는 2017년 구글의 "Attention is All You Need" 논문에서 등장한 모델로, 현대 NLP(자연어 처리)의 기틀이 된 아주 중요한 모델입니다. (BERT, GPT 모두 이 친구의 후손이죠!)

## 🏭 비유: 번역 공장 (Translation Factory)
Transformer 모델을 하나의 거대한 **'자동 번역 공장'**이라고 상상해봅시다.

1.  **원자재 입고 (Input Embedding)**: 영어 문장이라는 원자재가 들어옵니다. 기계가 이해할 수 있게 숫자(벡터)로 바뀝니다.
2.  **시간표 찍기 (Positional Encoding)**: 이 공장은 컨베이어 벨트가 없습니다(RNN처럼 순서대로 처리하지 않음). 그래서 자재마다 "나는 1번째 단어", "나는 2번째 단어"라는 시간표(순서 정보)를 붙여줍니다.
3.  **분석 부서 (Encoder)**: 
    - 들어온 문장을 꼼꼼히 분석합니다. 
    - "이 단어는 저 단어랑 관련이 있네?" (Self-Attention)
    - 분석한 내용을 요약해서 디코더에게 넘깁니다.
4.  **조립 부서 (Decoder)**: 
    - 인코더가 준 분석 보고서(Encoder Output)를 참고합니다.
    - 지금까지 만든 번역문을 다시 보면서(Masked Attention) 다음 단어를 하나씩 조립합니다.
5.  **출하 (Output Linear)**: 조립된 단어들을 최종 언어(한국어) 단어 중 하나로 선택해서 내보냅니다.

## 1. 공장 설비 설정 (Hyperparameter)
공장을 짓기 전에 설계도를 봅니다. 이 수치들은 모델의 크기와 성능을 결정합니다.

- **$d_{model} = 512$**: 각 단어(자재)가 가질 정보량의 크기입니다. (임베딩 차원)
- **$num\_layers = 6$**: 분석 부서(Encoder)와 조립 부서(Decoder)를 몇 층이나 쌓을지 결정합니다. 6층짜리 공장이네요.
- **$num\_heads = 8$**: 한 번에 몇 군데를 집중해서 볼지(Attention Head) 정합니다. 직원 8명이 동시에 분석한다고 보면 됩니다.
- **$d_{ff} = 2048$**: 각 층 내부의 개별 처리 공간(Feed Forward) 크기입니다.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

# 모델 설정값 (Hyperparameters)
d_model = 512              # 임베딩 차원 (단어 벡터의 크기)
num_layers = 6             # 층(Layer)의 개수 (Encoder, Decoder 각각)
num_heads = 8              # 멀티 헤드 어텐션의 헤드 개수
d_k = d_model // num_heads # 각 헤드가 담당할 차원 크기 (512 / 8 = 64)
d_ff = 2048                # 피드 포워드 신경망의 은닉층 크기
drop_out_rate = 0.1        # 드롭아웃 비율 (과적합 방지)

# 단어장 크기 (가정)
src_vocab_size = 10000     # 원문(Source) 단어 개수
trg_vocab_size = 10000     # 번역문(Target) 단어 개수

batch_size = 64            # 배치 크기
seq_len = 128              # 문장의 최대 길이

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 2. 위치 정보 주입 (Positional Encoding)
RNN은 단어를 순서대로 넣지만, 트랜스포머는 문장 전체를 한번에 '쾅'하고 넣습니다. 그래서 단어들이 원래 어디에 있었는지 모릅니다. 
**"너는 첫 번째, 너는 두 번째..."** 라고 알려주는 명찰표가 필요하죠.

단순히 1, 2, 3... 숫자를 더하면 값이 너무 커져서, **사인(sin)과 코사인(cos) 주기 함수**를 이용해 -1~1 사이의 독특한 패턴 값을 만들어 더해줍니다. 마치 시계바늘처럼 규칙적으로 돌아가는 값을 이용하는 원리입니다.

In [2]:
class PositionalEncoding(nn.Module):
    def __init__(self, seq_len, d_model):
        super().__init__()

        # (seq_len, d_model) 크기의 0 행렬 생성
        pe = torch.zeros(seq_len, d_model)
        
        # 위치(pos) 인덱스 생성 (0, 1, ..., seq_len-1). (seq_len, 1) 형태로 변환.
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        
        # 주기를 결정하는 분모 계산 (10000 ^ (2i / d_model))
        # log 공간에서 계산하여 수치 안정성 확보
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 짝수 인덱스(2i)에는 sin 함수 적용
        pe[:, 0::2] = torch.sin(position * div_term)
        # 홀수 인덱스(2i+1)에는 cos 함수 적용
        pe[:, 1::2] = torch.cos(position * div_term)

        # 배치 차원 추가 (1, seq_len, d_model)
        pe = pe.unsqueeze(0)
        
        # 학습되는 파라미터가 아니므로 buffer로 등록 (optimizer가 업데이트 안 함)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # x: (batch_size, seq_len, d_model)
        x = x * math.sqrt(d_model) # 임베딩 값 스케일링 (위치정보에 묻히지 않게)
        
        # 입력 길이에 맞춰 잘라서 더해주기
        x = x + self.pe[:, :x.size(1), :]
        return x

## 3. 품질 관리팀 (Layer Normalization)
공장 각 단계마다 데이터 값이 너무 커지거나 작아지지 않게 **정규화(Normalization)**를 해줍니다. 
데이터의 평균을 0, 분산을 1로 맞춰서 학습이 안정적으로 되게 돕는 '품질 관리팀' 같은 역할입니다.

In [3]:
class LayerNormalization(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        # 학습 가능한 파라미터 (scale, shift) - 감마, 베타
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        # 마지막 차원(d_model)에 대해 평균과 분산 구하기
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        
        # 정규화 및 스케일링
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

## 4. 개별 연구실 (Feed Forward Network)
Multi-Head Attention이 "팀원들과 회의"하는 것이라면, FFN은 "혼자 자리에 앉아서 정리"하는 시간입니다.
각 단어(위치)마다 개별적으로 적용되는 간단한 신경망(Linear -> ReLU -> Linear)입니다.

In [4]:
class FeedForwardLayer(nn.Module):
    def __init__(self, d_model, d_ff, drop_out_rate):
        super().__init__()
        # 1. 차원 확장 (512 -> 2048)
        self.linear1 = nn.Linear(d_model, d_ff)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop_out_rate)
        # 2. 차원 복구 (2048 -> 512)
        self.linear2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

## 5. 팀 회의 (Multi-Head Attention)
이 모델의 핵심! 단어들끼리 서로 정보를 주고받는 곳입니다. 
여러 개의 헤드(Head)가 각자 다른 관점(문법, 의미, 위치 등)에서 단어 사이의 관계를 파악합니다.

*   **Query**: 질문하는 주체
*   **Key**: 답변할 수 있는 대상
*   **Value**: 그 대상이 가진 정보

`attention()` 함수에서 Scaled Dot-Product Attention을 수행하고, `MultiHeadAttention` 클래스에서 입력 분할 및 병합을 처리합니다.

In [5]:
class MultiheadAttention(nn.Module):
    def __init__(self, d_model, drop_out_rate):
        super().__init__()
        self.num_heads = num_heads
        self.d_k = d_k # 헤드 당 차원

        # Q, K, V 생성을 위한 선형층
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        
        self.w_o = nn.Linear(d_model, d_model)  # 최종 출력 선형층
        self.dropout = nn.Dropout(drop_out_rate)
        self.softmax = nn.Softmax(dim=-1)

    def attention(self, q, k, v, mask=None):
        # 1. 점수 계산 (Query와 Key의 유사도)
        # (B, h, T, d_k) * (B, h, d_k, T) -> (B, h, T, T)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)

        # 2. 마스킹 적용 (필요할 경우)
        if mask is not None:
            # score가 매우 작은 수(-1e9)로 만들어 softmax 확률이 0이 되게 함
            # mask가 0인 부분이 가려질 부분 (구현 방식에 따라 1/0 의미는 다를 수 있음)
            # 여기선 mask가 0인 위치를 -1e9로 채움
            if mask.dim() == 2:
                mask = mask.unsqueeze(1).unsqueeze(1)
            scores = scores.masked_fill(mask == 0, -1e9)

        # 3. 확률 분포 (Softmax)
        attn_weights = self.softmax(scores)
        attn_weights = self.dropout(attn_weights)

        # 4. 가중합 (Value 섞기)
        output = torch.matmul(attn_weights, v)
        return output

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)

        # 1. 선형 변환
        q = self.w_q(q)
        k = self.w_k(k)
        v = self.w_v(v)

        # 2. 헤드 분할 (view & transpose)
        # (B, T, d_model) -> (B, T, h, d_k) -> (B, h, T, d_k)
        q = q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        k = k.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        v = v.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # 3. 어텐션 수행
        output = self.attention(q, k, v, mask)

        # 4. 헤드 병합 (transpose & view)
        # (B, h, T, d_k) -> (B, T, h, d_k) -> (B, T, d_model)
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, d_model)

        # 5. 최종 선형 변환 (섞기)
        output = self.w_o(output)
        return output

## 6. 인코더 레이어 (Encoder Layer)
분석 부서의 작업자는 두 가지 일을 합니다.
1.  **Multi-Head Attention**: 주변 단어들과 소통하며 문맥 파악 (팀 회의)
2.  **Feed Forward**: 파악된 내용을 혼자 정리 (개별 학습)

그리고 **Residual Connection (잔차 연결)**을 통해, 처리 전의 원래 정보도 잊지 않고 챙겨갑니다. (`x + sublayer(x)`)

In [6]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.mha = MultiheadAttention(d_model, drop_out_rate)
        self.ffn = FeedForwardLayer(d_model, d_ff, drop_out_rate)

        self.layernorm1 = LayerNormalization(d_model)
        self.layernorm2 = LayerNormalization(d_model)
        
        self.dropout1 = nn.Dropout(drop_out_rate)
        self.dropout2 = nn.Dropout(drop_out_rate)

    def forward(self, x, mask):
        # 1. Multi-Head Attention (Self-Attention)
        # 인코더에선 Q, K, V가 모두 자기 자신(x)입니다.
        attn_output = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output)
        # Residual Connection + Norm
        out1 = self.layernorm1(x + attn_output)

        # 2. Feed Forward Network
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        # Residual Connection + Norm
        out2 = self.layernorm2(out1 + ffn_output)

        return out2

class Encoder(nn.Module):
    def __init__(self, d_model, num_layers):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        # EncoderLayer를 6층(num_layers) 쌓음
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(num_layers)])

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return x

## 7. 디코더 레이어 (Decoder Layer)
조립 부서에서는 조금 더 복잡한 일을 합니다.

1.  **Masked Self-Attention**: 지금까지 쓴 자기 자신의 글을 되돌아봅니다. 단, **미래의 단어를 미리 컨닝하면 안 되므로** (`Look-ahead Mask`) 가려두고 봅니다. 마치 일기를 쓸 때, 아직 안 쓴 뒷내용은 참고할 수 없는 것과 같죠.
2.  **Cross-Attention (Encoder-Decoder Attention)**: 인코더 분석팀이 보내준 자료(Key, Value)를 참고해서 내 할 말(Query)을 만듭니다. "아까 그 문맥이 이거였지?" 하고 확인하는 과정입니다.
3.  **Feed Forward**: 정리 시간.

In [7]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.mha1 = MultiheadAttention(d_model, drop_out_rate)  # Masked Self-Attention
        self.mha2 = MultiheadAttention(d_model, drop_out_rate)  # Cross-Attention
        self.ffn = FeedForwardLayer(d_model, d_ff, drop_out_rate)

        self.layernorm1 = LayerNormalization(d_model)
        self.layernorm2 = LayerNormalization(d_model)
        self.layernorm3 = LayerNormalization(d_model)
        
        self.dropout1 = nn.Dropout(drop_out_rate)
        self.dropout2 = nn.Dropout(drop_out_rate)
        self.dropout3 = nn.Dropout(drop_out_rate)

    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        # 1. Masked Self-Attention (자신만 보기 + 미래 가리기)
        attn1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1)
        out1 = self.layernorm1(x + attn1)

        # 2. Cross-Attention (인코더 출력 참고)
        # Q: 디코더(자신), K: 인코더 출력, V: 인코더 출력
        attn2 = self.mha2(out1, enc_output, enc_output, padding_mask)
        attn2 = self.dropout2(attn2)
        out2 = self.layernorm2(out1 + attn2)

        # 3. Feed Forward
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output)
        out3 = self.layernorm3(out2 + ffn_output)

        return out3

class Decoder(nn.Module):
    def __init__(self, d_model, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(num_layers)])

    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        for layer in self.layers:
            x = layer(x, enc_output, look_ahead_mask, padding_mask)
        return x

## 8. 공장 가동 (Transformer Main)
이제 모든 부품을 조립해 완성된 Transformer 모델을 만듭니다.

In [8]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, seq_len, num_layers):
        super().__init__()
        # 1. 임베딩 + 위치 정보
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.trg_embedding = nn.Embedding(trg_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(seq_len, d_model)

        # 2. 인코더 & 디코더 스택
        self.encoder = Encoder(d_model, num_layers)
        self.decoder = Decoder(d_model, num_layers)

        # 3. 최종 출력층 (단어장 크기로 변환)
        self.output_layer = nn.Linear(d_model, trg_vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, src_inputs, trg_inputs, src_mask=None, trg_mask=None):
        # --- Encoder Part ---
        # 1. 소스 임베딩 + 위치 정보
        src_emb = self.src_embedding(src_inputs) # (B, T_src) -> (B, T_src, d_model)
        src_emb = self.positional_encoding(src_emb)

        # 2. 인코더 통과
        # enc_outputs: 인코더가 분석한 문맥 정보 (디코더의 K, V로 쓰임)
        enc_outputs = self.encoder(src_emb, src_mask) 

        # --- Decoder Part ---
        # 3. 타겟 임베딩 + 위치 정보
        trg_emb = self.trg_embedding(trg_inputs) # (B, T_trg) -> (B, T_trg, d_model)
        trg_emb = self.positional_encoding(trg_emb)

        # 4. 디코더 통과
        # 인코더 출력(enc_outputs)을 받아서 처리
        dec_outputs = self.decoder(trg_emb, enc_outputs, trg_mask, src_mask)

        # --- Output Part ---
        # 5. 다음 단어 확률 예측
        logits = self.output_layer(dec_outputs) # (B, T_trg, vocab_size)
        log_probs = self.softmax(logits)
        
        return log_probs

## 9. 모델 테스트 (Dummy Data)
모델이 잘 조립되었는지 가짜 데이터를 넣어서 확인해봅니다.

In [9]:
# 모델 생성
model = Transformer(src_vocab_size, trg_vocab_size, d_model, seq_len, num_layers)
model = model.to(device)

# 더미 데이터 생성 (Batch=64, Seq_Len=128)
src_input = torch.randint(0, src_vocab_size, (batch_size, seq_len)).to(device)
trg_input = torch.randint(0, trg_vocab_size, (batch_size, seq_len)).to(device)

# 모델 실행
model.eval() # 평가 모드
with torch.no_grad():
    output = model(src_input, trg_input)

print("===== 모델 입출력 크기 확인 =====")
print(f"Source Input Shape: {src_input.shape} (Batch, SeqLen)")
print(f"Target Input Shape: {trg_input.shape} (Batch, SeqLen)")
print(f"Model Output Shape: {output.shape} (Batch, SeqLen, VocabSize)")
print("성공적으로 모델이 동작합니다! 🎉")